## 1.1.2. Will Numba work for my code?

This depends on what your code looks like, if your code is numerically orientated (does a lot of math), uses NumPy a lot and/or has a lot of loops, then Numba is often a good choice. In these examples we’ll apply the most fundamental of Numba’s JIT decorators, `@jit`, to try and speed up some functions to demonstrate what works well and what does not.  

Numba works well on code that looks like this:

In [24]:
from numba import jit
import numpy as np

x = np.arange(100).reshape(10, 10)

def go_fast(a):
    trace = 0
    for i in range(len(a)):         # Numba likes loops
        trace += np.tanh(a[i, i])   # Numba likes NumPy functions
    return a + trace                # Numba likes NumPy broadcasting

go_fast2 = jit(go_fast, nopython=True)  # Another pattern of @njit on `go_fast`, easy to compare

In [16]:
%timeit -r 10 -n 5 go_fast(x)

27 µs ± 4.52 µs per loop (mean ± std. dev. of 10 runs, 5 loops each)


In [26]:
%timeit -r 10 -n 5 go_fast2(x)

The slowest run took 4.61 times longer than the fastest. This could mean that an intermediate result is being cached.
1.8 µs ± 1.41 µs per loop (mean ± std. dev. of 10 runs, 5 loops each)


It won’t work very well, if at all, on code that looks like this:

In [28]:
from numba import jit
import pandas as pd

x = {'a': [1, 2, 3], 'b': [20, 30, 40]}

def use_pandas(a): # Function will not benefit from Numba jit
    df = pd.DataFrame(a)           # Numba doesn't know about pd.DataFrame
    df += 1                        # Numba doesn't understand what this is
    return df.cov()                # or this!

use_pandas2 = jit(use_pandas)

In [30]:
%timeit -r 10 -n 5 use_pandas(x)

1.13 ms ± 191 µs per loop (mean ± std. dev. of 10 runs, 5 loops each)


In [33]:
%timeit -r 10 -n 5 use_pandas2(x)

1.21 ms ± 155 µs per loop (mean ± std. dev. of 10 runs, 5 loops each)


Note that Pandas is not understood by Numba and as a result Numba would simply run this code via the interpreter but with the added cost of the Numba internal overheads!